## Work With Data From the CIA World Factbook

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth.

In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [6]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Return the following:

- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [8]:
%%sql

select min(population), max(population), min(population_growth), max(population_growth)
from facts
;

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics on the previous screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)
Let's use subqueries to zoom in on just these countries.

In [18]:
%%sql

select name
from facts 
where population in (
    select min(population)
    from facts    
    )
;

Done.


name
Antarctica


In [17]:
%%sql

select name
from facts 
where population in (
    select max(population)
    from facts    
    )
;

Done.


name
World


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for Antarctica.

Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

In [14]:
%%sql

select min(population), max(population), min(population_growth), max(population_growth)
from facts
where name not in ('Antarctica', 'World')
;

Done.


min(population),max(population),min(population_growth),max(population_growth)
48,1367485388,0.0,4.02


Average value for the following columns:

- population
- area

In [47]:
%%sql

select avg(population), avg(area)
from facts
where name not in ('Antarctica', 'World')
;

Done.


avg(population),avg(area)
32377011.0125,555093.546184739


Countries meeting both of the following criteria:
- The population is above average.
- The area is below average.

In [48]:
%%sql
select name 
from facts
where area < (select avg(area) from facts)
and   population > (select avg(population) from facts where name not in ('Antarctica', 'World'))
;

Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


Which country has the most people? Which country has the highest growth rate?

In [49]:
%%sql

select name, population
from facts 
where population in (
    select max(population)
    from facts    
    where name not in ('Antarctica', 'World')
    )
;

Done.


name,population
China,1367485388


In [50]:
%%sql

select name, population_growth
from facts 
where population_growth in (
    select max(population_growth)
    from facts 
    where name not in ('Antarctica', 'World')
    )
;

Done.


name,population_growth
South Sudan,4.02


Which countries have the highest ratios of water to land? Which countries have more water than land?

In [51]:
%%sql

select name, area_water/area_land as ratio_water_land
from facts 
where ratio_water_land = (
    select max(area_water/area_land)
    from facts    
    where name not in ('Antarctica', 'World')
    )
;



Done.


name,ratio_water_land
British Indian Ocean Territory,905


In [53]:
%%sql

select name, area_water/area_land as ratio_water_land
from facts 
where ratio_water_land > 1
and name not in ('Antarctica', 'World')
;

Done.


name,ratio_water_land
British Indian Ocean Territory,905
Virgin Islands,4


Which countries will add the most people to their populations next year?

In [54]:
%%sql

select name, population_growth * population as next_year_population_growth
from facts 
where next_year_population_growth in (
    select max( population_growth * population)
    from facts   
    where name not in ('Antarctica', 'World')
    )
;

Done.


name,next_year_population_growth
India,1527068612.48


Which countries have a higher death rate than birth rate?

In [56]:
%%sql

select name, death_rate / birth_rate death_to_birth_ratio
from facts 
where name not in ('Antarctica', 'World')
and death_to_birth_ratio > 1
order by death_to_birth_ratio desc
;

Done.


name,death_to_birth_ratio
Bulgaria,1.6188340807174888
Serbia,1.5044052863436124
Latvia,1.431
Lithuania,1.4128712871287128
Hungary,1.3897379912663756
Monaco,1.3894736842105262
Slovenia,1.350356294536817
Ukraine,1.3488805970149254
Germany,1.3482880755608027
Saint Pierre and Miquelon,1.3099730458221026


Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [57]:
%%sql

select name, population/area as population_area_ratio 
from facts
where name not in ('Antarctica', 'World')
order by population_area_ratio desc
limit 20
;

Done.


name,population_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


In [58]:
%%sql


select name 
from facts
where area < (select avg(area) from facts)
and   population > (select avg(population) from facts)
and name not in ('Antarctica', 'World')
and name in 
(
select name
from facts
order by population/area desc
limit 20
)
;

Done.


name
Bangladesh


Only Bangladesh is in the top 20 of countries having the biggest ratio population to area and an area above the average area of the countries.